In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle('./data/raw_data.pkl', compression='zip')

In [3]:
# Filter: Drop companies whose cogs, xsga, txt and oiadp are missing
data = data.dropna(subset=['cogs', 'xsga', 'txt', 'oiadp'])
# Filter: Top 1000 companies by total assets
data = data.sort_values(['fyear', 'at'], ascending=[True, False]).groupby('fyear').head(1000)
# Filter: We have set the condition that indfmt = 'INDL' in the sql query

In [4]:
# Calculation: exp = (sale - ib) / at
# rev = sale / at
# earn = rev - exp
# cogs, sga, depr, tax, oiadp, si = ~.x/at
# oth = exp - cogs - sga - depr - tax - si
# depr equals 0 if missing
# si_indicator = 1 if si >= 0.01, otherwise 0
data['exp'] = (data['sale'] - data['ib']) / data['at']
data['rev'] = data['sale'] / data['at']
data['earn'] = data['rev'] - data['exp']
data[['cogs', 'sga', 'depr', 'tax', 'oiadp', 'si']] = data[['cogs', 'xsga', 'dp', 'txt', 'oiadp', 'spi']].div(data['at'], axis=0)
data['si'] = -data['si']
data['oth'] = data['exp'] - data['cogs'] - data['sga'] - data['depr'] - data['tax'] - data['si']
data.loc[data['depr'].isna(), 'depr'] = 0
data['si_indicator'] = np.where(data['si'] >= 0.01, 1, 0)

In [5]:
data = data.sort_values(['gvkey', 'fyear'])
# Calculation: Average 2-year revenue
data['avg_2yr_rev'] = data.groupby('gvkey')['rev'].rolling(window=2).mean().reset_index(level=0, drop=True)
# Calculation: 10-year volat of 2-year revenue
data['rev_10yr_volat'] = data.groupby('gvkey')['avg_2yr_rev'].rolling(window=10).std().reset_index(level=0, drop=True)
# Calculation: Average 2-year expense
data['avg_2yr_exp'] = data.groupby('gvkey')['exp'].rolling(window=2).mean().reset_index(level=0, drop=True)
# Calculation: 10-year volat of 2-year expense
data['exp_10yr_volat'] = data.groupby('gvkey')['avg_2yr_exp'].rolling(window=10).std().reset_index(level=0, drop=True)
# Calculation: Average 2-year earnings
data['avg_2yr_earn'] = data.groupby('gvkey')['earn'].rolling(window=2).mean().reset_index(level=0, drop=True)
# Calculation: 5-year and 10-year volat of 2-year earnings
data['earn_5yr_volat'] = data.groupby('gvkey')['earn'].rolling(window=5).std().reset_index(level=0, drop=True)
data['earn_10yr_volat'] = data.groupby('gvkey')['avg_2yr_earn'].rolling(window=10).std().reset_index(level=0, drop=True)
# Calculation: Last-year expense, next-year expense and last-year earnings
data['earn_lag'] = data.groupby('gvkey')['earn'].shift(1)
data['exp_lag'] = data.groupby('gvkey')['exp'].shift(1)
data['exp_lead'] = data.groupby('gvkey')['exp'].shift(-1)

In [6]:
# Winsorization: 1%
vars_winsor = [
    'avg_2yr_rev', 'avg_2yr_exp', 'avg_2yr_earn',
    'rev_10yr_volat', 'exp_10yr_volat', 'earn_5yr_volat', 'earn_10yr_volat',
    'rev', 'exp', 'exp_lag', 'exp_lead', 'earn', 'earn_lag'
]
def winsorize(group):
    for var in vars_winsor:
        q_low = group[var].quantile(0.01)
        q_high = group[var].quantile(0.99)
        group[var] = group[var].clip(q_low, q_high)
    return group
data = data.groupby('fyear').apply(winsorize).reset_index(drop=True)

In [7]:
len(data)

69537

In [8]:
# Filter: Keep the obs. with 2-year average earnings, 5-year and 10-year earnings volatility
data = data.dropna(subset=['avg_2yr_earn', 'earn_5yr_volat', 'earn_10yr_volat'])

In [9]:
# Select variables
data = data[['gvkey', 'fyear', 'at', 'rev', 'exp', 'exp_lag', 'exp_lead', 'earn', 'earn_lag', 'avg_2yr_rev', 'avg_2yr_exp', 'avg_2yr_earn', 'rev_10yr_volat', 'exp_10yr_volat', 'earn_5yr_volat', 'earn_10yr_volat', 'cogs', 'sga', 'depr', 'tax', 'si', 'si_indicator', 'oiadp', 'oth']]

In [10]:
data.to_csv('./data/master_data.csv', index=False)